# Model Comparison and Selection

This notebook compares all trained models and selects the best performing one for salary prediction with F1 score > 0.90.


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib
from sklearn.metrics import f1_score, roc_auc_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


## 1. Load Data and Models


In [ ]:
# Load test data
test_data = pd.read_csv('./for_cursur/Data/salary.test.processed.csv', index_col='id')
live_data = pd.read_csv('./for_cursur/Data/salary.live.processed.csv', index_col='id')

X_test = test_data.drop(columns=['label'])
y_test = test_data['label']
X_live = live_data

print(f"Test data shape: {X_test.shape}")
print(f"Live data shape: {X_live.shape}")
print(f"Test target distribution: {y_test.value_counts().to_dict()}")

# Load all model configurations
model_configs = {}
model_files = [
    ('XGBoost', './for_cursur/xgb_model_config.json'),
    ('LightGBM', './for_cursur/lgb_model_config.json'),
    ('CatBoost', './for_cursur/catboost_model_config.json'),
    ('Random Forest', './for_cursur/rf_model_config.json'),
    ('Ensemble', './for_cursur/ensemble_model_config.json')
]

print("\nLoading model configurations...")
for model_name, config_file in model_files:
    try:
        with open(config_file, 'r') as f:
            config = json.load(f)
            model_configs[model_name] = config
            print(f"✅ {model_name}: F1 = {config.get('test_f1_score', 'N/A'):.4f}")
    except FileNotFoundError:
        print(f"❌ {model_name}: Config file not found")
    except Exception as e:
        print(f"❌ {model_name}: Error loading config - {e}")


## 2. Model Performance Comparison


In [ ]:
# Create comparison dataframe
comparison_data = []
for model_name, config in model_configs.items():
    comparison_data.append({
        'Model': model_name,
        'F1_Score': config.get('test_f1_score', 0),
        'AUC_Score': config.get('test_auc_score', 0),
        'CV_Mean_F1': config.get('cv_mean_f1', 0),
        'Feature_Count': config.get('feature_count', 0),
        'Training_Samples': config.get('training_samples', 0),
        'Test_Samples': config.get('test_samples', 0),
        'Target_Achieved': config.get('test_f1_score', 0) >= 0.90
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('F1_Score', ascending=False)

print("=== MODEL PERFORMANCE COMPARISON ===")
print(comparison_df.to_string(index=False, float_format='%.4f'))

# Find best model
best_model = comparison_df.iloc[0]
print(f"\n🏆 BEST MODEL: {best_model['Model']}")
print(f"   F1 Score: {best_model['F1_Score']:.4f}")
print(f"   AUC Score: {best_model['AUC_Score']:.4f}")
print(f"   Target Achieved: {'✅ YES' if best_model['Target_Achieved'] else '❌ NO'}")

# Count models that achieved target
target_achieved = comparison_df['Target_Achieved'].sum()
print(f"\n📊 Models achieving F1 > 0.90: {target_achieved}/{len(comparison_df)}")


## 3. Performance Visualization


In [ ]:
# Create performance comparison plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# F1 Score comparison
axes[0, 0].bar(comparison_df['Model'], comparison_df['F1_Score'], color='skyblue', alpha=0.7)
axes[0, 0].axhline(y=0.90, color='red', linestyle='--', label='Target F1 = 0.90')
axes[0, 0].set_title('F1 Score Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('F1 Score')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# AUC Score comparison
axes[0, 1].bar(comparison_df['Model'], comparison_df['AUC_Score'], color='lightgreen', alpha=0.7)
axes[0, 1].set_title('AUC Score Comparison', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('AUC Score')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# F1 vs AUC scatter plot
colors = ['red' if not achieved else 'green' for achieved in comparison_df['Target_Achieved']]
axes[1, 0].scatter(comparison_df['F1_Score'], comparison_df['AUC_Score'], 
                   c=colors, s=100, alpha=0.7)
axes[1, 0].axvline(x=0.90, color='red', linestyle='--', alpha=0.5, label='Target F1 = 0.90')
axes[1, 0].set_xlabel('F1 Score')
axes[1, 0].set_ylabel('AUC Score')
axes[1, 0].set_title('F1 Score vs AUC Score', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Add model labels to scatter plot
for i, model in enumerate(comparison_df['Model']):
    axes[1, 0].annotate(model, (comparison_df.iloc[i]['F1_Score'], comparison_df.iloc[i]['AUC_Score']),
                       xytext=(5, 5), textcoords='offset points', fontsize=8)

# Target achievement pie chart
target_counts = comparison_df['Target_Achieved'].value_counts()
labels = ['Target NOT Achieved', 'Target Achieved'] if False in target_counts.index else ['Target Achieved']
sizes = [target_counts.get(False, 0), target_counts.get(True, 0)] if False in target_counts.index else [target_counts.get(True, 0)]
colors_pie = ['lightcoral', 'lightgreen'] if len(sizes) == 2 else ['lightgreen']

axes[1, 1].pie(sizes, labels=labels, colors=colors_pie, autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title('Target Achievement Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Print detailed performance summary
print("\n=== DETAILED PERFORMANCE SUMMARY ===")
for _, row in comparison_df.iterrows():
    status = "✅ TARGET ACHIEVED" if row['Target_Achieved'] else "❌ Target not achieved"
    print(f"{row['Model']:15s}: F1={row['F1_Score']:.4f}, AUC={row['AUC_Score']:.4f} - {status}")


## 4. Load Best Model and Make Final Predictions


In [ ]:
# Load the best model
best_model_name = best_model['Model']
print(f"Loading best model: {best_model_name}")

# Load the appropriate model file
model_file_map = {
    'XGBoost': './for_cursur/xgb_model.joblib',
    'LightGBM': './for_cursur/lgb_model.joblib',
    'CatBoost': './for_cursur/catboost_model.cbm',
    'Random Forest': './for_cursur/rf_model.joblib',
    'Ensemble': './for_cursur/ensemble_model.joblib'
}

try:
    if best_model_name == 'CatBoost':
        from catboost import CatBoostClassifier
        best_model_loaded = CatBoostClassifier()
        best_model_loaded.load_model(model_file_map[best_model_name])
    else:
        best_model_loaded = joblib.load(model_file_map[best_model_name])
    
    print(f"✅ Successfully loaded {best_model_name} model")
    
    # Make predictions on live data
    live_predictions = best_model_loaded.predict(X_live)
    live_probabilities = best_model_loaded.predict_proba(X_live)[:, 1]
    
    # Create final prediction dataframe
    final_predictions = pd.DataFrame({
        'id': X_live.index,
        'predicted_label': live_predictions,
        'probability_high_income': live_probabilities,
        'model_used': best_model_name
    })
    
    print(f"\n🎯 FINAL PREDICTIONS COMPLETED!")
    print(f"   Model Used: {best_model_name}")  
    print(f"   F1 Score: {best_model['F1_Score']:.4f}")
    print(f"   Number of predictions: {len(final_predictions)}")
    print(f"   High income predictions: {live_predictions.sum()}")
    print(f"   Low income predictions: {len(live_predictions) - live_predictions.sum()}")
    
    # Save final predictions
    final_predictions.to_csv('./for_cursur/final_predictions.csv', index=False)
    print(f"\n💾 Final predictions saved to: ./for_cursur/final_predictions.csv")
    
    # Show sample predictions
    print(f"\n📋 Sample predictions:")
    print(final_predictions.head(10))
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Please ensure all model files are properly trained and saved.")


## 5. Summary and Recommendations


In [ ]:
# Final summary
print("=" * 60)
print("🎯 SALARY PREDICTION MODEL SELECTION SUMMARY")
print("=" * 60)

print(f"\n📊 TOTAL MODELS TESTED: {len(comparison_df)}")
print(f"✅ MODELS ACHIEVING F1 > 0.90: {target_achieved}")
print(f"🏆 BEST MODEL: {best_model['Model']}")
print(f"📈 BEST F1 SCORE: {best_model['F1_Score']:.4f}")
print(f"📈 BEST AUC SCORE: {best_model['AUC_Score']:.4f}")

print(f"\n🔍 MODEL RANKINGS:")
for i, (_, row) in enumerate(comparison_df.iterrows(), 1):
    status = "✅" if row['Target_Achieved'] else "❌"
    print(f"   {i}. {row['Model']:15s} - F1: {row['F1_Score']:.4f} {status}")

print(f"\n💡 RECOMMENDATIONS:")
if best_model['Target_Achieved']:
    print(f"   ✅ SUCCESS! {best_model['Model']} achieved your target F1 > 0.90")
    print(f"   🚀 Use {best_model['Model']} for production predictions")
    print(f"   📁 Final predictions saved in: ./for_cursur/final_predictions.csv")
else:
    print(f"   ⚠️  No model achieved F1 > 0.90")
    print(f"   🔧 Consider:")
    print(f"      - More feature engineering")
    print(f"      - Different hyperparameter tuning")
    print(f"      - Ensemble methods")
    print(f"      - More training data")

print(f"\n📋 NEXT STEPS:")
print(f"   1. Run individual model notebooks to train all models")
print(f"   2. Run this comparison notebook to select the best model")
print(f"   3. Use final_predictions.csv for your salary predictions")
print(f"   4. Monitor model performance in production")

print("\n" + "=" * 60)
